# **Analyse des tendances de prix des ordinateurs portables sur eBay**

### **Introduction**


Notre projet vise à étudier les tendances de consommation des utilisateurs de plateformes de e-commerce, en se concentrant sur les ordinateurs portables sur eBay. Les questions centrales de cette étude sont les suivantes :
- Quels sont les facteurs qui influencent la demande d’un produit et, par conséquent, son prix ?
- Comment estimer le prix d’un ordinateur en fonction de ses caractéristiques ?

Pour y répondre, nous avons distingué deux types de critères :

**Critères objectifs :** RAM, stockage, densité de pixels, condition de l’article.

**Critères subjectifs :** marque, couleur, taille de l’écran.

Nous avons trouvé le choix de la plateforme eBay pertinent dans la mesure où ce sont les utilisateurs qui fixent le prix qu'ils attribuent à leur ordinateur. Cette dynamique permet d’évaluer si les vendeurs sont objectifs dans leur valorisation du produit ou si le prix fixé est cohérent avec les caractéristiques de l’ordinateur. Grâce aux tendances observées, il devient possible de déterminer si ces prix reflètent réellement la valeur des produits.

Afin d’analyser ces influences, nous avons choisi de modéliser la relation entre les caractéristiques des produits et leur prix à l’aide d’une régression linéaire. Cette approche permet de quantifier l’impact de chaque critère sur le prix des ordinateurs portables. Nous pourrons ainsi mieux comprendre comment les utilisateurs déterminent le prix de vente de leurs ordinateurs portables et savoir si certaines variables les influencent plus que d'autres. 


### **Installation**

In [5]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement re (from versions: none)
ERROR: No matching distribution found for re


In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


ModuleNotFoundError: No module named 'utils'

### **Chargement des données dans le projet**

En raison du temps de réponse élevé de l’API et du nombre d'appels à l'API limité à 4500 par jour, nous avons préféré exécuter les requêtes en amont et sauvegarder les données dans un fichier CSV. Cette démarche a pour objectif de faciliter l’analyse en évitant les délais d’attente liés à l’API et en permettant d'avoir plus de données. 

Le code permettant de collecter les données est disponible sur le GitHub du projet. Ce script **data.py** situé dans le fichier **src** peut être exécuté en suivant les instructions du README pour l'accès à l'API si de nouvelles données doivent être extraites.

Dans le notebook, nous nous contentons de charger le fichier CSV prégénéré pour effectuer les étapes d’analyse et de nettoyage.

Cette approche permet une expérience plus fluide et garantit la reproductibilité de l’étude sans dépendre des performances de l’API au moment de l’exécution.



### **Récupération des données**

Pour collecter les données nécessaires à notre étude, nous avons utilisé l’API Browse d’eBay. Ce choix s’explique par la richesse des informations disponibles sur cette plateforme et son large catalogue de produits. Cependant, cela a requis plusieurs étapes préparatoires :

1. **Choix d'un segment spécifique :** Pour limiter la variabilité des données, nous avons choisi de nous concentrer sur les ordinateurs portables. Ce segment présente de nombreuses spécificités (RAM, stockage, écran, etc.) qui permettent d’étudier l’impact de chaque critère sur le prix.

2. **Délimitation du marché :** Nous avons choisi de nous focaliser sur le marché français. Cette décision a pour but d’éviter les biais liés à des contextes géopolitiques différents entre les marchés ou à des devises différentes. 

3. **Extraction des données :**
Les informations classiques comme le prix et l’état des articles sont directement accessibles via l’API Browse pour chaque item.
Les caractéristiques plus précises (RAM, marque, stockage, taille de l’écran) sont stockées dans une sous-catégorie nommée localized aspects. Ces informations sont organisées sous forme de dictionnaire (clé/valeur).

4. **Gestion des données manquantes :**
Comme les utilisateurs remplissent eux-mêmes les informations, certaines données sont parfois absentes. 
Plusieurs approches ont été envisagées :
- Exclure les articles avec des données manquantes, en raison du volume important de données disponibles.
- Conserver les articles incomplets pour ne pas réduire excessivement l’échantillon.

Afin de ne pas trop réduire l'échantillon ou de compromettre la fiabilité de l'étude nous avons choisi de conserver les articles ayant obligatoirement les informations suivantes : prix, capacité de stockage, RAM et marque. Toutefois, si des données telles que la taille de l'écran, la résolution ou encore la date de publication ne sont pas présentes il ne nous semblait pas pertinent d'enlever ces ordinateurs portables puisque cela ne compromet pas le résultat de notre étude et nous permet d'avoir un échantillon plus important.

In [ ]:
ordi_csv = "data3.csv"
df = pd.read_csv(ordi_csv)
df.head()

,ID,Prix,Condition,RAM,Stockage,Marque,Couleur,Taille écran,Résolution,Date de publication,Rang
0,v1|176737201854|0,1680.0,Occasion,16 Go,0,ASUS,NaN,"13,4""",NaN,2024-12-15T17:33:06.000Z,5.0
1,v1|375863586836|0,550.0,Occasion,8 Go,256 Go,Apple,Gris,"13""",2560 x 1600,2024-12-13T08:42:48.000Z,1.0
2,v1|146269671817|0,120.0,Occasion,6 Go,700Go,ASUS,Rouge,"15,6""",NaN,2024-12-15T11:51:11.000Z,5.0
3,v1|196898961056|0,89.9,Occasion,8 Go,480Go,HP,NaN,"17,3""",NaN,2024-12-17T16:37:02.000Z,3.0
4,v1|365291982473|0,1300.0,Occasion,16 Go,256 Go,Dell,Noir,"13""",1920 x 1080,2024-12-17T16:57:08.000Z,7.0


### **Nettoyage des données**

Comme évoqué plus tôt, la particularité des données récoltées est qu'elles ont été entrées par les vendeurs eux-mêmes via la plateforme. Ainsi, si le nom de la catégorie (ex: 'RAM', 'Stockage') est le même pour chaque produit, les informations contenues dans chaque colonne différent dans leur format. Par défaut, toutes les données sont de type "object" ce qui ne les rend pas exploitable directement pour nous. Le travail de nettoyage est ainsi essentiel afin d'avancer sur notre projet.  

1) **Nettoyage des données chiffrées**

Pour cela, nous avons créé une fonction **extract_float_from_object** au sein du fichier **utils.py** qui permet d'extraire un nombre de type float à partir d'une chaîne de caractère. Nous pouvons appliquer cette fonction directement pour la catégorie 'Taille écran'. 

In [13]:
from src.utils import extract_float_from_object

def extract_taille_ecran(df):
    df['Taille écran'] = df['Taille écran'].apply(extract_float_from_object)
    return df

Pour gérer les catégories liées au stockage dont les données étaient soit rentrées en Go, soit en To, nous avons créé une seconde fonction dérivée de la première, mais qui effectue la conversion en Go pour les données entrées en To, selon la formule **1To = 1024Go**.

In [21]:
from src.utils import extract_storage

def clean_giga_columns(df):
    df['RAM'] = df['RAM'].apply(extract_storage)
    df['Stockage'] = df['Stockage'].apply(extract_storage)
    return df

Ensuite, pour gérer la colonne résolution dont le format est le suivant : **Largeur x Hauteur'**, nous avons choisi de combiner cette information avec la taille de l'écran pour établir une mesure comparable entre tous les ordinateurs du data frame : la densité de pixels (PPI).

$$
Densité\ de\ pixels = \frac{\sqrt{\text{largeur}^2 + \text{hauteur}^2}}{\text{taille de l'écran en pouces}}
$$

On commence par extraire la largeur et la hauteur dans deux nouvelles colonnes, et on utilise ensuite la formule ci-dessus pour créer la colonne **PPI** que l'on conservera.

In [22]:
def extract_resolution(df):
    df[['Largeur', 'Hauteur']] = df['Résolution'].str.extract(r'(\d+)[\s]*[xX][\s]*(\d+)', expand=True)
    df[['Largeur', 'Hauteur']] = df[['Largeur', 'Hauteur']].apply(pd.to_numeric, errors='coerce')
    return df

def calculate_ppi(df):
    df['Taille écran'] = df['Taille écran'].apply(extract_float_from_object)
    df[['Largeur', 'Hauteur', 'Taille écran']] = df[['Largeur', 'Hauteur', 'Taille écran']].replace(0, np.nan)
    # On calcule le PPI uniquement pour les lignes où toutes les valeurs nécessaires sont présentes
    mask = df[['Largeur', 'Hauteur', 'Taille écran']].notnull().all(axis=1)
    df.loc[mask, 'PPI'] = np.round(np.sqrt(df.loc[mask, 'Largeur']**2 + df.loc[mask, 'Hauteur']**2) / df.loc[mask, 'Taille écran'])
    return df

2. **Nettoyage des données temporelles**

Cela concerne uniquement la colonne **Date de publication**. Ici, le format est idéal car c'est une information qui nous est fournie directement par eBay. On utilise alors la fonction intégrée à pandas **pd.to_datetime()**

In [17]:
def format_date(df):
    df['Date de publication'] = pd.to_datetime(df['Date de publication'], errors='coerce')
    return df


3. **Nettoyage des données textuelles**

Cela concerne les colonnes **Condition**, **Marque** et **Couleur**. Ici, l'enjeu est d'uniformiser les données afin de rassembler toutes les données identiques de fait mais rentrées différemment par les utilisateurs. 

In [23]:
#Normalisation des couleurs
def normalize_color(color):
    if isinstance(color, str):
        color = color.lower()
        if 'gris' in color or 'silver' in color or 'argent' in color or 'argenté' in color or 'grey' in color and 'graphite' in color:
            return 'gris'
        elif 'noir' in color or 'black' in color:
            return 'noir'
        elif 'blanc' in color or 'white' in color:
            return 'blanc'
        elif 'bleu' in color or 'midnight' in color or 'blue' in color:
            return 'bleu'
        elif 'rouge' in color or 'red' in color:
            return 'rouge'
        elif 'vert' in color or 'green' in color:
            return 'vert'
        elif 'jaune' in color or 'yellow' in color:
            return 'jaune'
        elif 'rose' in color or 'pink' in color:
            return 'rose'
        elif 'marron' in color or 'brown' in color:
            return 'marron'
        elif 'violet' in color or 'purple' in color:
            return 'violet'
    return "autre"

def clean_color_column(df):
    df['Couleur'] = df['Couleur'].apply(normalize_color)
    return df

#Normalisation des conditions 
def convertir_condition(condition):
    """ 
    Crée une classification des conditions des produits du meilleur au pire
    """
    if "Neuf" in condition:
        return "Neuf"
    elif "Ouvert (jamais utilisé)" in condition:
        return "Ouvert"
    elif 'Parfait état - Reconditionné' in condition: 
        return "Parfait état"
    elif 'Très bon état - Reconditionné' in condition:
        return "Très bon état"
    elif 'État correct - Reconditionné' in condition:
        return "État correct"
    elif 'Occasion' in condition: 
        return "Occasion"
    
def clean_condition(df):
    df['Condition'] = df['Condition'].apply(convertir_condition)
    return df

#Normalisation des marques
def format_marque(marque):
    if isinstance(marque, str):
        marque = marque.lower()
        if any(substring in marque for substring in ['carte graphique', 'nvidia']):
            return np.nan #Ici, on élimine également les lignes qui ne correspondent 
                          #pas à des ordinateurs portables, rentrées par erreur
        elif 'apple' in marque or 'macbook' in marque:
            return 'Apple'
        elif 'dell' in marque or 'del' in marque:
            return 'Dell'
        elif 'hp' in marque:
            return 'HP'
        elif 'lenovo' in marque:
            return 'Lenovo'
        elif 'asus' in marque:
            return 'Asus'
        elif 'acer' in marque:
            return 'Acer'
        elif 'samsung' in marque:
            return 'Samsung'
        elif 'sony' in marque:
            return 'Sony'
        elif 'toshiba' in marque:
            return 'Toshiba'
        elif 'huawei' in marque:
            return 'Huawei'
        elif 'msi' in marque:
            return 'MSI'
        elif 'panasonic' in marque:
            return 'Panasonic'
        elif 'microsoft' in marque:
            return 'Microsoft'
        elif 'lg' in marque:
            return 'LG'
        elif 'google' in marque:
            return 'Google'
        elif 'alienware' in marque:
            return 'Alienware'
        elif 'razer' in marque:
            return 'Razer'
        elif 'gigabyte' in marque:
            return 'Gigabyte'
        elif 'clevo' in marque:
            return 'Clevo'
        elif 'fujitsu' in marque:
            return 'Fujitsu'
        elif 'medion' in marque:
            return 'Medion'
        elif 'xmg' in marque:
            return 'XMG'
        elif 'chuwi' in marque:
            return 'Chuwi'
        elif 'jumper' in marque:
            return 'Jumper'
        elif 'teclast' in marque:
            return 'Teclast'
        elif 'voyo' in marque:
            return 'Voyo'
        elif 'bmax' in marque:
            return 'BMAX'
        elif 'one-netbook' in marque:
            return 'One-Netbook'
        elif 'gpd' in marque:
            return 'GPD'
        elif 'tuxedo' in marque:
            return 'Tuxedo'
        elif 'system76' in marque:
            return 'System76'
        elif 'purism' in marque:
            return 'Purism'
        elif 'pine64' in marque:
            return 'Pine64'
        elif 'minisforum' in marque:
            return 'Minisforum'
        elif 'azulle' in marque:
            return 'Azulle'
        elif 'beelink' in marque:
            return 'Beelink'
        elif 'meego' in marque:
            return 'Meego'
        elif 'vorke' in marque:
            return 'Vorke'
        elif 'trigkey' in marque:
            return 'Trigkey'
        elif 'acepc' in marque:
            return 'ACEPC'
        elif 'awow' in marque:
            return 'AWOW'
        elif 'niuniutab' in marque:
            return 'Niuniutab'
        else:
            return np.nan
    return np.nan

def clean_brand_column(df):
    df['Marque'] = df['Marque'].apply(format_marque)
    df = df.dropna(subset=['Marque'])
    return df

On peut à présent exécuter toutes ces fonctions afin de générer un nouveau data frame exploitable **data_semicleaned.csv**.


In [24]:
df = clean_giga_columns(df)
df = clean_color_column(df)
df = extract_resolution(df)
df = extract_taille_ecran(df)
df = convertir_condition(df)
df = calculate_ppi(df)
df = format_date(df)
df = clean_brand_column(df)
df = format_date(df)
df = df.drop(['Largeur', 'Hauteur', 'Résolution'], axis=1)
#la colonne résolution est remplacée par la colonne PPI qui compile taille de l'écran et résolution

df.to_csv('data_semicleaned.csv', index=False) 
#génération d'un nouveau csv pour les données formatées 

TypeError: 'NoneType' object is not subscriptable

Pour obtenir notre data frame final, il reste encore à gérer les valeurs aberrantes qui sont assez fréquentes du fait des erreurs ou mauvais choix des utilisateurs eBay.

4. **Gestion des valeurs aberrantes**

Un bon outil pour repérer les valeurs aberrantes est la boîte à moustache qui nous donne une vision claire de la répartition des données pour chaque catégorie. On peut la générer pour chaque colonne grâce à la fonction suivante:

In [ ]:
df = pd.read_csv('data_semicleaned.csv')

def boxplot(df, column_name):
    plt.figure(figsize=(10, 6))
    sns.boxplot(x=df[column_name])
    plt.title(f'Boxplot of {column_name}')
    plt.xlabel(column_name)
    plt.show()

boxplot(df, 'Prix')
#boxplot(df, 'Stockage')
#boxplot(df, 'RAM')
#boxplot(df, 'PPI')
#boxplot(df, 'Taille écran')

Grâce à ces boites à moustache ainsi qu'un peu de logique, on établit une plage cohérente pour chaque colonne. Toute donnée à l'extérieur de ces plages est considérée comme "aberrante" et est gérée en fonction. 

**Caractéristiques des plages** :

- **Plage des prix** : $[50\ € ;\ 3000\ €]$
- **Plage de la densité de pixels** : $[80\ \text{PPI} ;\ 300\ \text{PPI}]$
- **Plage de la capacité de stockage** : $[32\ \text{Go} ;\ 4000\ \text{Go}]$
- **Plage de la capacité de RAM** : $[2\ \text{Go} ;\ 64\ \text{Go}]$
- **Plage de la taille d’écran** : $[6'' ;\ 20'']$


In [ ]:
df = df[(df['Prix'] >= 50) & (df['Prix'] <= 3000)]  
df.loc[(df['PPI'] > 300) | (df['PPI'] < 80), 'PPI'] = np.nan
# énormément de gens se trompent en mettant la même valeur pour la RAM et le stockage
df.loc[(df['Stockage'] < 32) | (df['Stockage'] > 4000), 'Stockage'] = np.nan  
df.loc[(df['RAM'] < 2) | (df['RAM'] > 64), 'RAM'] = np.nan  
df.loc[(df['Taille écran'] < 6) | (df['Taille écran'] > 20), 'Taille écran'] = np.nan  


Etant donné que l'on cherche à estimer le prix, on a décidé de supprimer les lignes où les prix étaient aberrants. Pour les autres variables, on a décidé de les remplacer par des **NaN** pour les exclure de la visualisation sans se priver de l'information que nous apporte les autres données de cet ordinateur.

On peut à présent générer notre data frame nettoyé

In [ ]:
df.to_csv('data_cleaned.csv', index=False)

### **Visualisation**

### **Modélisation**